In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import seaborn as sns
import h5py
import tqdm
import zipfile
from tqdm.notebook import tqdm_notebook
from progress.bar import IncrementalBar
import random
import pandas as pd
import time
import os
import math
from joblib import Parallel, delayed
# tf.config.run_functions_eagerly(True)
%matplotlib inline

In [2]:
param_names=['signal','pl_fr','real_wf-pl_fr','mask']

In [3]:
def norming(data,param_names=param_names,log=True):
    
    # ADD LOGARIFM FOR SIGNAL
    norm_params={}
    for i in range(data.shape[-1]):
        if (i==0 and log):
            data[:,:,:,i]=tf.math.log(data[:,:,:,i]+1)
        max_c=data[:,:,:,i].max()
        min_c=data[:,:,:,i].min()
        mean_c=data[:,:,:,i].mean()
        std_c=data[:,:,:,i].std()
        print(max_c,min_c)
        data[:,:,:,i]=(data[:,:,:,i]-min_c)/(max_c-min_c)

        norm_params[param_names[i]]=np.array([max_c,min_c])
    return norm_params
def renormin(data_not_change,norm_params,param_names=param_names,log=True):
    data=np.copy(data_not_change)
    for i in range(data.shape[-1]):
        max_c, min_c=norm_params[param_names[i]]
        data[:,:,:,i]=data[:,:,:,i]*(max_c-min_c) + min_c
        if (i==0 and log):
            data[:,:,:,i]=tf.math.exp(data[:,:,:,i])-1
        elif (i==1 or i==2):
            data[:,:,:,i]=data[:,:,:,i]*1e6
    return data

In [4]:
path='../../mc_hadrons_qgs34_0010.h5'
data_name='pr-q4-9yr'
data=np.zeros((1,6,6,4))
with h5py.File(path,'r') as f:
    data=np.concatenate((data,f[data_name]['dt'][:100,:,:,3:7]))
    gp=f[data_name]['gp'].value
    ev_params=f[data_name]['ev_params'].value
data=data[1:]
data[:,:,:,1]=data[:,:,:,1]*1e6
data[:,:,:,2]=data[:,:,:,2]*1e6
print('len data: ',len(data))

len data:  100


/home/rfit/.conda/envs/robert_venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  
/home/rfit/.conda/envs/robert_venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  import sys


# Constant

In [5]:
pi = tf.constant(math.pi)
UNIT=1
dist=tf.constant(1.2,dtype=tf.float32)# min dist between 2 detectors in km
c=tf.constant(299792.458,dtype=tf.float32)# # km\s
NSEC= 1e6/c# in rubsov's code is a time for 1.2 km 1.2/c*1e9
R_L=30e-3
LINSLEY_r0=0.025
Det_Area=30
tf_type=tf.float32
s_min = tf.constant([[0.3]])
s_max = tf.constant([[1.8]])
t_err_res=c/1e6
t0_err=30

In [41]:
class Reconstruction():
    #constants
    pi = tf.constant(math.pi)
    UNIT=1
    dist=tf.constant(1.2,dtype=tf.float32)# min dist between 2 detectors in km
    c=tf.constant(299792.458,dtype=tf.float32)# # km\s
    NSEC= 1e6/c# in rubsov's code is a time for 1.2 km 1.2/c*1e9
    R_L=30e-3
    LINSLEY_r0=0.025
    Det_Area=3
    s_min = tf.constant([[0.3]])
    s_max = tf.constant([[1.8]])
    t_err_res=c/1e6
    t0_err=30
    
    def __init__(self,data,tf_type):
        # all have shape (batch,6,6,1)
        self.tf_type = tf_type
        self.data=tf.cast(data,self.tf_type)
        self.mask = tf.expand_dims(data[:,:,:,3],-1)
        self.signal = tf.expand_dims(data[:,:,:,0],-1)*self.mask
        self.real_time = tf.expand_dims((data[:,:,:,1] + data[:,:,:,2]),-1)*self.mask 
        self.signal = tf.cast(self.signal,self.tf_type)
        self.real_time = tf.cast(self.real_time,self.tf_type)
        self.mask = tf.cast(self.mask,self.tf_type)
        self.batch = data.shape[0]
        self.t0=None
        self.theta=None
        self.detertors_coord=None
        self.core=None
        self.t_place=None
#     @classmethod
    def detectors_init(self):
        x = tf.cast(tf.repeat(tf.expand_dims(tf.range(0,6),0),6,axis=0),self.tf_type)
        y = tf.cast(tf.repeat(tf.expand_dims(tf.range(0,6),1),6,axis=1),self.tf_type)
        x = tf.expand_dims(x,-1)
        y = tf.expand_dims(y,-1)
        # x.shape = (6,6,1)
        detectors = tf.concat([x,y],axis=-1) * dist
        
        #detectors.shape = (batch,6,6,2) in km
        detectors = tf.repeat(tf.expand_dims(detectors,0),self.batch,0)
        self.detertors_coord=detectors
        return detectors
#     @classmethod
    def core_(self):
        detectors = self.detectors_init() # shape (batch,6,6,2)
        sum_signal=tf.reduce_sum(self.signal,axis=(1,2)) # shape (batch ,1)
        core=tf.reduce_sum(detectors * self.signal, axis=(1,2))/sum_signal
        #update self
        self.core = core
        return core
    @staticmethod
    def expand_dims(vec):
        return tf.expand_dims(tf.expand_dims(vec,-1),-1)
    def create_matrix(self,x,y,t,mask):
        a11=self.expand_dims(tf.reduce_sum(x*x,axis=1))
        a12=self.expand_dims(tf.reduce_sum(x*y,axis=1))
        a13=self.expand_dims(tf.reduce_sum(x,axis=1))
        a22=self.expand_dims(tf.reduce_sum(y*y,axis=1))
        a23=self.expand_dims(tf.reduce_sum(y,axis=1))
        a33=self.expand_dims(tf.reduce_sum(mask,axis=1))
        a1=tf.concat([a11,a12,a13],axis=2)
        a2=tf.concat([a12,a22,a23],axis=2)
        a3=tf.concat([a13,a23,a33],axis=2)
        A=tf.concat([a1,a2,a3],axis=1)

        b1=self.expand_dims(tf.reduce_sum(x*t,axis=1))
        b2=self.expand_dims(tf.reduce_sum(y*t,axis=1))
        b3=self.expand_dims(tf.reduce_sum(t,axis=1))
        b=tf.concat([b1,b2,b3],axis=1)
        return A,b
    def place_sol(self):
        real_time=self.real_time
        mask=self.mask
        detectors = self.detectors_init() * mask
        x = tf.reshape(detectors[:,:,:,0],(-1,36))
        y = tf.reshape(detectors[:,:,:,1],(-1,36))
        t = tf.reshape(real_time,(-1,36))
        mask = tf.reshape(mask,(-1,36))
        A,b=self.create_matrix(x,y,t,mask)
        return tf.linalg.solve(A,b)
#     @classmethod
    def place_params(self):
        sol = self.place_sol()
        #t_0=b+(r_cor;n)
        b=sol[:,2:,0]
        n=sol[:,:2,0]
        core = self.core_()
        mul=n[:,0]*core[:,0]+n[:,1]*core[:,1]
#         t0 = tf.cast(tf.expand_dims(b,1)+tf.expand_dims(mul,1),dtype=self.tf_type)
        t0=b
        a_x=n[:,0]
        a_y=n[:,1]
        a_z=tf.math.pow(1e6/c,2) - (tf.math.pow(a_x,2) + tf.math.pow(a_y,2))
        a_z=tf.where(a_z>0,tf.math.sqrt(a_z),0)
        cos_theta=a_z*(c/1e6)
        theta=tf.math.acos(cos_theta)
        tg_phi=a_x/(-a_y)
        atan=tf.math.atan(tg_phi)
        phi=tf.where(a_y>0,atan+pi,atan-pi)
        phi=tf.where(a_x<0,atan,phi)
        phi = tf.math.atan2( -a_x, -a_y )
        # update self
        self.phi = phi
        self.theta = theta
        self.t0=t0
        return t0,tf.expand_dims(theta,-1),tf.expand_dims(phi,-1)
    def detectors_core(self):
        detectors=self.detertors_coord
        core = self.core
        detectors_c=detectors-tf.reshape(core,(-1,1,1,2))
        return detectors_c
#     @classmethod
    def place_reconstruction(self):
        t0,theta,phi = self.place_params()
        detectors = self.detectors_core()
        detectors = self.detertors_coord
#         print(detectors)
        t0=self.expand_dims(t0) # shape (batch,1,1,1)
        theta=self.expand_dims(theta)
        phi = self.expand_dims(phi)
        n=-tf.concat([tf.math.cos(phi)*tf.math.sin(theta),tf.math.sin(phi)*tf.math.sin(theta)],axis=-1)*(1e6/c)
#         print(n)
        t_place =  t0 + detectors[:,:,:,1:2]*n[:,:,:,0:1] + detectors[:,:,:,0:1]*n[:,:,:,1:2]
        self.t_place = t_place*self.mask
        return self.t_place
#     def

In [42]:
rec = Reconstruction(data[:10],tf.float32)
det=rec.detectors_init()
core=rec.core_()
sol = rec.place_sol() 
# sol

In [43]:
rec.place_reconstruction()[:10,:,:,0]

<tf.Tensor: shape=(10, 6, 6), dtype=float32, numpy=
array([[[-0.        , -0.        , -0.        , -0.        ,
         -0.        , -0.        ],
        [ 0.        , -0.        , -0.        , -0.        ,
         -0.        , -0.        ],
        [ 0.        ,  0.        , -0.        , -3.1249313 ,
         -5.158413  , -0.        ],
        [ 0.        ,  0.        ,  0.93723774, -1.0962429 ,
         -3.1297245 , -0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.9324465 ,
         -0.        , -0.        ],
        [ 0.        ,  0.        ,  4.9946156 ,  0.        ,
          0.        , -0.        ]],

       [[-0.        , -0.        , -0.        , -0.        ,
         -0.        ,  0.        ],
        [-0.        , -0.        , -0.        , -0.        ,
         -0.        ,  0.        ],
        [-0.        , -0.        , -0.        , -0.9901347 ,
          0.        ,  0.        ],
        [-0.        , -0.        , -1.8163371 ,  0.19841337,
          2.

In [21]:
tf.reduce_max(tf.math.abs(rec.place_reconstruction()[:10,:,:,0]-data[:10,:,:,1]),axis=(1,2))

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([ 1.1361599, 25.09468  , 32.694958 , 10.307863 ,  1.2820619,
        8.549691 , 22.997522 , 17.645132 ,  7.5969834, 12.285401 ],
      dtype=float32)>

In [116]:
rec.place_reconstruction()[:10,:,:,0]

<tf.Tensor: shape=(10, 6, 6), dtype=float32, numpy=
array([[[-0.        , -0.        , -0.        , -0.        ,
         -0.        , -0.        ],
        [ 0.        , -0.        , -0.        , -0.        ,
         -0.        , -0.        ],
        [ 0.        ,  0.        , -0.        , -3.0157812 ,
         -5.04447   , -0.        ],
        [ 0.        ,  0.        ,  1.0463886 , -0.9823002 ,
         -3.0109892 , -0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.0511808 ,
         -0.        , -0.        ],
        [ 0.        ,  0.        ,  5.1133504 ,  0.        ,
          0.        , -0.        ]],

       [[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  2.6260128 ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  1.7998109 ,  0.6112624 ,
         -0.

In [20]:
# reconstruction with defined params
t0,theta,phi = rec.place_params()
theta = tf.expand_dims(ev_params[:10,2] ,-1)
phi = tf.expand_dims(ev_params[:10,3]  ,-1)
detectors = rec.detectors_core()
t0=rec.expand_dims(t0) # shape (batch,1,1,1)
theta=rec.expand_dims(theta)
phi = rec.expand_dims(phi)
n=tf.concat([tf.math.cos(phi)*tf.math.sin(theta),tf.math.sin(phi)*tf.math.sin(theta)],axis=-1)*(1e6/c)
t_place = detectors[:,:,:,0:1]*n[:,:,:,0:1] + detectors[:,:,:,1:2]*n[:,:,:,1:2]
t_place = t_place * rec.mask
t_place[8,:,:,0]

<tf.Tensor: shape=(6, 6), dtype=float32, numpy=
array([[-0.        , -0.        , -0.        , -0.        ,  0.        ,
         0.        ],
       [-0.        , -0.        , -0.        , -0.        ,  0.        ,
         0.        ],
       [-0.        , -0.        , -0.        , -0.39722878,  0.        ,
         0.        ],
       [-0.        , -6.588819  , -2.9601529 ,  0.6685133 ,  4.297179  ,
         0.        ],
       [-0.        , -5.523077  , -1.8944108 ,  1.7342553 ,  0.        ,
         0.        ],
       [-0.        , -0.        , -0.        ,  0.        ,  0.        ,
         0.        ]], dtype=float32)>

In [21]:
data[8,:,:,1]

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        , -2.05800006,  0.        ,
         0.        ],
       [ 0.        ,  2.81799998,  0.79699998, -1.17900004, -3.17600006,
         0.        ],
       [ 0.        ,  3.69500003,  1.69500004, -0.34000001,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ]])

In [22]:
ev_params[:10,2:4] 

array([[ 42.4385, 131.57  ],
       [ 40.6731, 245.529 ],
       [ 38.2434,  24.3915],
       [ 38.1833,  87.2113],
       [ 41.7928, 136.063 ],
       [ 46.2211, 298.839 ],
       [ 31.3009, 200.646 ],
       [ 32.9256,  18.2795],
       [ 32.653 , 113.383 ],
       [ 30.5313,  80.6433]], dtype=float32)

In [23]:
tf.where(rec.phi*180/3.14>0,rec.phi*180/3.14,(rec.phi+3.14*2)*180/3.14)

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([135.00084 , 239.4015  ,  28.259281, 113.2955  , 139.18312 ,
       289.5204  , 205.41646 ,  20.930283, 114.04648 ,  62.362896],
      dtype=float32)>

In [82]:
rec.theta*180/3.14

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([45.879997, 35.77847 , 48.284664, 30.336348, 38.42599 , 26.98432 ,
       34.245453, 21.895771, 23.444355, 13.708007], dtype=float32)>

In [85]:
data[0,:,:,1]

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        , -3.59299997, -5.57600015,
         0.        ],
       [ 0.        ,  0.        ,  0.325     , -1.71399995, -3.74799993,
         0.        ],
       [ 0.        ,  0.        ,  0.        , -0.125     ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  3.89200022,  0.        ,  0.        ,
         0.        ]])

In [3]:
num = 64
h5f = '../../mc_hadrons_qgs34_0010.h5'
h5f = '/home3/ivkhar/TA/data/mass_spectrum/old_1223/hadrons/bundled/mc_hadrons_qgs34_0010.h5'
with h5py.File(h5f,'r') as hf:
    times_flat = hf['pr-q4-9yr/dt'][:num,:,:,4:5]*1e6 # in mks
    times_diff = hf['pr-q4-9yr/dt'][:num,:,:,5:6]*1e6 # in mks
    mask = hf['pr-q4-9yr/dt'][:num,:,:,6:7]
    real_coords = hf['pr-q4-9yr/dt'][:num,:,:,:3] # in 1.2*km
    qs = hf['pr-q4-9yr/dt'][:num,:,:,3:4]
    ev_params = hf['pr-q3-9yr/ev_params'][:num]
    
times_reg = times_flat + times_diff
mask_qs = tf.where( tf.math.logical_and(mask>0, qs>0), 1., 0. )
xs = tf.expand_dims( tf.linspace( 0., 5*1.2, 6 ), axis=-1 )
xs = tf.repeat(xs, 6, axis=1)
ys = tf.transpose( xs )
coords = tf.expand_dims( tf.stack( (xs,ys), axis=-1 ), axis=0 ) # in km
c = tf.constant(0.299792458, dtype=tf.float32) # in km\mks
# prepare matricies to solve
def make_matrix(coords, times_reg, mask):
    
    # matrix for dL/db
    hs = tf.ones((1,6,6,1)) # to make (x_1,x_2) -> (x_1,x_2,1), to account for b
    coords_ext = mask*tf.concat( (coords,hs), axis=-1 ) # (bs,6,6,3)
    matrix_b = tf.expand_dims( tf.math.reduce_sum( coords_ext, axis=(1,2) ), axis=-1 ) # (bs,6,6,3,1)
    times = mask*times_reg # (bs,6,6,1)
    rhs_b = tf.math.reduce_sum( times, axis=(1,2) ) # (bs,1)
    
    # matrix for dL/da
    coords_mul = tf.expand_dims( coords_ext, axis=-1) * tf.expand_dims( coords, axis=-2)
    matrix_a = tf.math.reduce_sum( coords_mul, axis=(1,2) ) # (bs,6,6,3,2)
    ts_mul = times * coords
    rhs_a = tf.math.reduce_sum( ts_mul, axis=(1,2) )
    
    matrix = tf.concat((matrix_b,matrix_a), axis=-1)
    rhs = tf.expand_dims( tf.concat((rhs_b,rhs_a), axis=-1), axis=-1 )
    
    return matrix, rhs
def lin_solve(matrix, rhs):
    sol_flat = tf.linalg.solve( matrix, rhs, adjoint=True )[:,:,0]
    print(sol_flat)
    t0 = sol_flat[:,2]
    # extract parameters
    n_z = 1/c/c - tf.math.reduce_sum(sol_flat[:,:2]*sol_flat[:,:2], axis=1)
    n_z = tf.where( n_z>0, tf.math.sqrt( n_z ), 0. ) # some configurations yield front moving faster than light
    print('n_z',n_z)
    theta_flat = tf.math.acos( n_z*c )
    phi_flat = tf.math.atan2( sol_flat[:,1], sol_flat[:,0] )
    
    return theta_flat, phi_flat, t0
matrix, rhs = make_matrix(coords, times_reg, mask)
sol_flat=lin_solve(matrix, rhs)
theta, phi, t0 = lin_solve(matrix, rhs) # (bs,)
phi/3.14*180


tf.Tensor(
[[  1.690568    -1.6945748   -1.0818156 ]
 [  0.9904436    1.678957    -9.411423  ]
 [ -2.192621    -1.1778892   12.180873  ]
 [  0.66441846  -1.5473608    3.3125598 ]
 [  1.5665901   -1.3564273   -2.033852  ]
 [ -0.50638616   1.4255589   -5.9170237 ]
 [  1.693558     0.80760324  -7.2979884 ]
 [ -1.1613629   -0.44394162   7.119641  ]
 [  0.539285    -1.2119058    2.6467204 ]
 [ -0.36687446  -0.6997201    4.3824077 ]
 [ -0.8971767   -0.7005648    6.763143  ]
 [  0.8620558   -1.7710629    4.004331  ]
 [  1.486538     1.1685897   -8.332845  ]
 [  0.44323626  -1.4122609    5.4817886 ]
 [ -1.2110292   -0.6453041    9.464282  ]
 [ -0.47500038   1.054167    -1.446     ]
 [ -1.126339    -1.322737     9.62021   ]
 [ -1.9163822    0.02383728   8.648997  ]
 [ -0.8919314    1.9643096   -1.9128022 ]
 [  0.8607137    1.9327376   -7.716711  ]
 [  0.58633155  -1.741169     4.4970164 ]
 [ -1.4055791    0.8666418    3.4287083 ]
 [ -1.594655     1.0198795    2.3299828 ]
 [ -0.32000008   0.8293

<tf.Tensor: shape=(64,), dtype=float32, numpy=
array([ -45.090675 ,   59.493122 , -151.83202  ,  -66.795746 ,
        -40.908295 ,  109.61158  ,   25.507902 , -159.16087  ,
        -66.04494  , -117.72843  , -142.08736  ,  -64.07822  ,
         38.190773 ,  -72.61241  , -152.02588  ,  114.31395  ,
       -130.48126  ,  179.37828  ,  114.47934  ,   66.028435 ,
        -71.42547  ,  148.41838  ,  147.47336  ,  111.154564 ,
         99.09006  ,  111.39848  ,  -38.16081  , -173.6324   ,
        152.98344  ,   44.788975 ,  -45.31987  ,  -15.201984 ,
        138.35306  , -160.82552  ,   20.068356 ,  -95.30713  ,
         71.44137  , -154.94284  ,  107.48643  ,   48.002686 ,
          4.0393744,  -55.981457 ,  133.64186  , -174.19273  ,
         71.1309   ,  130.7888   ,  116.42755  ,  -37.481594 ,
         11.4989195,   85.55882  ,    1.0422605,  138.42151  ,
         57.84859  , -141.22237  ,   13.51256  ,  -13.543718 ,
       -105.87633  ,  178.18668  , -165.66962  ,  -29.445803 ,
        

In [162]:
sol[:2]

<tf.Tensor: shape=(2, 3, 1), dtype=float32, numpy=
array([[[-1.6945673 ],
        [ 1.690574  ],
        [-1.0818663 ]],

       [[ 1.6789589 ],
        [ 0.99045706],
        [-9.411484  ]]], dtype=float32)>

In [160]:
t0_ext = t0[:,tf.newaxis,tf.newaxis]
nx = 1/c*tf.math.sin(theta)[:,tf.newaxis,tf.newaxis] * tf.math.cos(phi)[:,tf.newaxis,tf.newaxis]
ny = 1/c*tf.math.sin(theta)[:,tf.newaxis,tf.newaxis] * tf.math.sin(phi)[:,tf.newaxis,tf.newaxis]
a = tf.concat((nx,ny), axis=-1)
a = tf.expand_dims( a, axis=1 )

t_rec = t0_ext + tf.math.reduce_sum( a*coords, axis=-1 )
t_rec = t_rec*mask[:,:,:,0]
times_flat[:,:,:,0]*mask[:,:,:,0]

array([[[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , -3.593     ,
         -5.576     ,  0.        ],
        [ 0.        ,  0.        ,  0.325     , -1.714     ,
         -3.748     ,  0.        ],
        [ 0.        ,  0.        ,  0.        , -0.125     ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  3.8920002 ,  0.        ,
          0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , -1.665     ,
          0.        ,  0.        ],
        [ 0.        ,  0.        , -2.3170002 , -0.62      ,
          1.778     ,  0.        ],
        [ 0.        ,  0.   